In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import redditutils as ru
import word2vecReader as wvr

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import recall_score

from skopt import BayesSearchCV


%load_ext autoreload
%autoreload 2

In [2]:
!ls

100k_response.pkl             random_forest.pkl
100k_train.pkl                reddit-ETL.ipynb
1k_features.pkl               redditutils.py
1k_response.pkl               results.csv
25k_test.pkl                  seinfeld-chronicles
25k_test_features.pkl         short_test_matrix.csv.mtx
Model_tuning.ipynb            short_train_matrix.csv.mtx
Project Fletcher Proposal.pdf shower_clean.csv
README.md                     showerthoughts-clean.ipynb
ROC_AUC.png                   showerthoughts.csv
__pycache__                   test-time.ipynb
bayes_search.pkl              test.csv
classification.ipynb          test_csv.csv.
cleaned_shower.csv            tfidf.ipynb
cleaning.csv                  tokenized.csv
count-vec-models.ipynb        train.csv
darkweb-EDA.ipynb             truth.pkl
final_df.csv                  vectorized_df
final_model.pkl               vectorized_df.csv
first_5k_response.pkl         w2v_mod.pkl
first_5k_words.pkl            word2vec.ipynb
fitted_cv.pkl               

In [3]:
df = pd.read_csv('final_df.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
X = df[['title', 'hour', 'dow']]
y = df['quality']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2325)

In [6]:
X_train_short = X_train[:100000]
X_test_short = X_test[:25000]

y_train_short = y_train[:100000]
y_test_short = y_test[:25000]

In [25]:
time = X_train_short['hour']

In [13]:
time = np.array(time)

In [14]:
np.where(np.isnan(time))

(array([29547]),)

In [18]:
np.median(time)

/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3250: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


nan

In [21]:
type(time[0])

numpy.float64

In [31]:
time[29547]

12.0

In [33]:
time[29547]

12.0

In [35]:
imp = Imputer(missing_values = np.nan, strategy = 'mean')

/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [40]:
time = imp.fit_transform(np.array(time).reshape(1, -1))

In [44]:
gb = GradientBoostingClassifier()

In [47]:
X_train_short['hour'] = X_train_short['hour'].fillna(X_train_short['hour'].mean())

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [54]:
X_train_short = X_train_short['hour'].astype(int)

In [60]:
pd.Series(X_train_short).astype(str)

986502    16
410051     9
366505    22
27722     10
773750    21
714648     2
738479    19
517313    14
866133    20
42053     15
145266    13
770240    21
607590    16
977959    16
634414    20
197698    12
31423     22
901764     3
866272    11
491297    21
571967    18
563087     3
628577    16
967205    17
900417     8
615511    23
287637    21
888694     3
731138     7
835172    15
          ..
816267    15
413797     1
757912     0
729855    12
479168    21
864986     7
574308    22
264761    17
260937     0
150626    14
540986     4
221873     5
559767    12
448814    18
87347      5
195368    20
525842     7
469907    14
298994    15
96427     18
60344     22
798771    15
427926     3
384828     0
633118     3
351536    11
814965    12
524625     3
814270    11
972914    15
Name: hour, Length: 100000, dtype: object

In [61]:
X_train_short = pd.Series(X_train_short).astype(str)

In [62]:
scores = cross_val_score(gb, pd.get_dummies(X_train_short), y_train_short, cv=3, scoring='roc_auc')

In [63]:
scores

array([0.51148299, 0.50620578, 0.5017081 ])

In [65]:
df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)